7.1  Create a crawler using the webcrawler provided and discussed in my jupyter notebook ( https://github.com/pschragger/big-data-python-class/blob/master/Lectures/Lecture_7_-_Link_Analysis.ipynb) , have it crawl the first 200 pages from a base URL of your choosing<br>
7.2  Create a Stochastic matrix from its resulting crawling as per https://cs7083.wordpress.com/2013/01/31/demystifying-the-pagerank-and-hits-algorithms/<br>
7.3 Pass it through the Page Rank algorithm and provide the list of the top 5 page URLs in your sample<br>
7.x For extra credit use the hits algorithm ( with a connectivity matrix )<br>
7.xx For advanced credit find another ranking alogrithm and demonstrate that as well<br>

## Creating first spider

In [1]:
%%writefile code/books_crawler/books_crawler/items.py
# %load code/tutorial/tutorial/items.py

# Define here the models for your scraped items

import scrapy

## Adding container to hold scraped data using scrapy.Item as the parent class
class BooksCrawlerItem(scrapy.Item):
    Inurl = scrapy.Field()
    Outurl = scrapy.Field()
    #url = scrapy.Field()
    pass


Overwriting code/books_crawler/books_crawler/items.py


In [5]:
%%writefile code/books_crawler/books_crawler/spiders/books.py

import scrapy
from scrapy.http import Request
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from books_crawler.items import BooksCrawlerItem

class BooksSpider(CrawlSpider):
    name = 'books'
    allowed_domains = ['books.toscrape.com']
    start_urls = ['http://books.toscrape.com',]
    
    rules = [Rule(LinkExtractor(canonicalize=True, unique=True), callback='parse_page')]

    for i in range(2, 50):
        start_urls.append("http://books.toscrape.com/catalogue/page-"+str(i)+".html")
        
    def parse_page(self, response):
        items = []
        links = LinkExtractor(canonicalize=True, unique=True).extract_links(response)
        for link in links:
            for dm in self.allowed_domains:
                if dm in link.url:
                    item = BooksCrawlerItem()
                    item['Inurl'] = response.url
                    item['Outurl'] = link.url
                    items.append(item)
                    yield item
        #items


Overwriting code/books_crawler/books_crawler/spiders/books.py


In [3]:
#cd code/books_crawler

C:\Users\Dinesh\Desktop\Git Repositories\big-data-python-class\Homeworks\Homework7\code\books_crawler


In [8]:
import requests
from scrapy.http import TextResponse

#running scrappy to get a response from a URL
r = requests.get('http://books.toscrape.com')
response = TextResponse(r.url, body=r.text, encoding='utf-8')

print(response)

<200 http://books.toscrape.com/>


In [9]:
#Using xpath to access data
response.xpath('//a')

[<Selector xpath='//a' data='<a href="index.html">Books to Scrape</a>'>,
 <Selector xpath='//a' data='<a href="index.html">Home</a>'>,
 <Selector xpath='//a' data='<a href="catalogue/category/books_1/inde'>,
 <Selector xpath='//a' data='<a href="catalogue/category/books/travel'>,
 <Selector xpath='//a' data='<a href="catalogue/category/books/myster'>,
 <Selector xpath='//a' data='<a href="catalogue/category/books/histor'>,
 <Selector xpath='//a' data='<a href="catalogue/category/books/sequen'>,
 <Selector xpath='//a' data='<a href="catalogue/category/books/classi'>,
 <Selector xpath='//a' data='<a href="catalogue/category/books/philos'>,
 <Selector xpath='//a' data='<a href="catalogue/category/books/romanc'>,
 <Selector xpath='//a' data='<a href="catalogue/category/books/womens'>,
 <Selector xpath='//a' data='<a href="catalogue/category/books/fictio'>,
 <Selector xpath='//a' data='<a href="catalogue/category/books/childr'>,
 <Selector xpath='//a' data='<a href="catalogue/category/books

In [1]:
import numpy as np
import pandas as pd

In [2]:
book_links = pd.read_csv("code/books_crawler/books_crawler/BookLinks.csv")

In [4]:
print(type(book_links))
book_links.head()

<class 'pandas.core.frame.DataFrame'>


,Inurl,Outurl
0,http://books.toscrape.com/catalogue/night-sky-...,http://books.toscrape.com/index.html
1,http://books.toscrape.com/catalogue/page-6.html,http://books.toscrape.com/index.html
2,http://books.toscrape.com/catalogue/night-sky-...,http://books.toscrape.com/catalogue/category/b...
3,http://books.toscrape.com/catalogue/night-sky-...,http://books.toscrape.com/catalogue/category/b...
4,http://books.toscrape.com/catalogue/night-sky-...,http://books.toscrape.com/catalogue/poems-that...


In [8]:
grpby_Inurls = book_links[['Inurl','Outurl']].groupby(['Inurl'],as_index=False)

In [10]:
bookLinks_grped = grpby_Inurls.count()

In [17]:
bookLinks_grped.head()

,Inurl,Outurl
0,http://books.toscrape.com/catalogue/10-day-gre...,9
1,http://books.toscrape.com/catalogue/10-happier...,9
2,http://books.toscrape.com/catalogue/112263_583...,9
3,http://books.toscrape.com/catalogue/13-hours-t...,9
4,http://books.toscrape.com/catalogue/1491-new-r...,9


In [18]:
bookLinks_grped.shape

(1082, 2)

In [22]:
H = np.zeros((1082,1082))

In [23]:
H

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [26]:
import random
for i in range (H.shape[0]):
    p = 1/float(bookLinks_grped.loc[[i]]['Outurl'])
    nos = bookLinks_grped.at[i,'Outurl']
    ls = [random.sample(range(H.shape[1]), nos)] 
    for j in ls:
        H[i][j]=p

In [27]:
H

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [30]:
print(bookLinks_grped.loc[[10]]['Outurl'])
print(H[10].max())
print(H[10].sum())

10    9
Name: Outurl, dtype: int64
0.111111111111
1.0


In [31]:
from numpy import *
 
def pagerank(H):
    n= len(H)
    w = zeros(n)
    rho = 1./n * ones(n);
    for i in range(n):
      if multiply.reduce(H[i]== zeros(n)):
        w[i] = 1
    newH = H + outer((1./n * w),ones(n))
 
    theta=0.85
    G = (theta * newH) + ((1-theta) * outer(1./n * ones(n), ones(n)))
    print (rho)
    for j in range(10):
        rho = dot(rho,G)
        print (rho)
        if j==9:
            return rho

In [32]:
ranks = pagerank(H)

[ 0.00092421  0.00092421  0.00092421 ...,  0.00092421  0.00092421
  0.00092421]
[ 0.000924    0.00080707  0.00135594 ...,  0.00144615  0.00066702
  0.00128867]
[ 0.00094816  0.00078288  0.00123723 ...,  0.00138338  0.00067431
  0.00127857]
[ 0.00098959  0.00077432  0.0012304  ...,  0.00136258  0.00068485
  0.00130325]
[ 0.0009751   0.00077264  0.00123517 ...,  0.00136302  0.00068676
  0.00131789]
[ 0.00097643  0.00077469  0.00123511 ...,  0.00136124  0.00068816
  0.00131636]
[ 0.00097625  0.00077472  0.00123427 ...,  0.00136128  0.00068826
  0.00131652]
[ 0.00097618  0.00077476  0.00123434 ...,  0.00136154  0.00068821
  0.00131677]
[ 0.00097621  0.00077476  0.00123435 ...,  0.00136147  0.00068822
  0.00131676]
[ 0.0009762   0.00077476  0.00123435 ...,  0.00136148  0.00068822
  0.00131677]
[ 0.0009762   0.00077476  0.00123435 ...,  0.00136148  0.00068822
  0.00131677]


In [33]:
bookLinks_grped.loc[np.argsort(ranks)[-10:]]['Inurl']

693     http://books.toscrape.com/catalogue/spark-joy-...
527     http://books.toscrape.com/catalogue/outliers-t...
395     http://books.toscrape.com/catalogue/lady-midni...
227     http://books.toscrape.com/catalogue/digital-fo...
664     http://books.toscrape.com/catalogue/shoe-dog-a...
145     http://books.toscrape.com/catalogue/category/b...
256     http://books.toscrape.com/catalogue/everydata-...
1003    http://books.toscrape.com/catalogue/tracing-nu...
492     http://books.toscrape.com/catalogue/neither-he...
629     http://books.toscrape.com/catalogue/romero-and...
Name: Inurl, dtype: object